In [5]:
# automatically reload changes from other modules without having to restart kernel
%load_ext autoreload
%autoreload 2

In [1]:
from Corpora import MovieReviewCorpus
from Lexicon import SentimentLexicon
from Statistics import SignTest
from Classifiers import NaiveBayesText, SVMText
from Extensions import SVMDoc2Vec

In [7]:
# retrieve corpus
corpus=MovieReviewCorpus(stemming=False,pos=False)

# use sign test for all significance testing
signTest=SignTest()

print("--- classifying reviews using sentiment lexicon  ---")

# read in lexicon
lexicon=SentimentLexicon()

# on average there are more positive than negative words per review (~7.13 more positive than negative per review)
# to take this bias into account will use threshold (roughly the bias itself) to make it harder to classify as positive
threshold=8

# question 0.1
lexicon.classify(corpus.reviews,threshold,magnitude=False)
token_preds=lexicon.predictions
print(f"token-only results: {lexicon.getAccuracy():.2f}")

lexicon.classify(corpus.reviews,threshold,magnitude=True)
magnitude_preds=lexicon.predictions
print(f"magnitude results:{lexicon.getAccuracy():.2f}")

# question 0.2
p_value=signTest.getSignificance(token_preds,magnitude_preds)
significance = "significant" if p_value < 0.05 else "not significant"
print(f"magnitude lexicon results are {significance} with respect to token-only")

--- classifying reviews using sentiment lexicon  ---
token-only results: 0.68
magnitude results:0.69
magnitude lexicon results are not significant with respect to token-only


In [8]:
# question 1.0
print("--- classifying reviews using Naive Bayes on held-out test set ---")
NB=NaiveBayesText(smoothing=False,bigrams=False,trigrams=False,discard_closed_class=False)
NB.train(corpus.train)
NB.test(corpus.test)
# store predictions from classifier
non_smoothed_preds=NB.predictions
print(f"Accuracy without smoothing: {NB.getAccuracy():.2f}")

--- classifying reviews using Naive Bayes on held-out test set ---
Accuracy without smoothing: 0.52


In [9]:
# question 2.0
# use smoothing
NB=NaiveBayesText(smoothing=True,bigrams=False,trigrams=False,discard_closed_class=False)
NB.train(corpus.train)
NB.test(corpus.test)
smoothed_preds=NB.predictions
# saving this for use later
num_non_stemmed_features=len(NB.vocabulary)
print(f"Accuracy using smoothing: {NB.getAccuracy():.2f}")


# question 2.1
# see if smoothing significantly improves results
p_value=signTest.getSignificance(non_smoothed_preds,smoothed_preds)
significance = "significant" if p_value < 0.05 else "not significant"
print(f"results using smoothing are {significance} with respect to no smoothing")

Accuracy using smoothing: 0.82
results using smoothing are significant with respect to no smoothing


In [10]:
# question 3.0
print("--- classifying reviews using 10-fold cross-evaluation ---")
# using previous instantiated object
NB.crossValidate(corpus)
# using cross-eval for smoothed predictions from now on
smoothed_preds=NB.predictions
print(f"Accuracy: {NB.getAccuracy():.3f}")
print(f"Std. Dev: {NB.getStdDeviation():.3f}")

--- classifying reviews using 10-fold cross-evaluation ---
Accuracy: 0.810
Std. Dev: 0.020736441353327688


In [11]:
# question 4.0
print("--- stemming corpus ---")
# retrieve corpus with tokenized text and stemming (using porter)
stemmed_corpus=MovieReviewCorpus(stemming=True,pos=False)
print("--- cross-validating NB using stemming ---")
NB.crossValidate(stemmed_corpus)
stemmed_preds=NB.predictions
print(f"Accuracy: {NB.getAccuracy():.3f}")
print(f"Std. Dev: {NB.getStdDeviation():.3f}")

# Q4.1
p_value=signTest.getSignificance(smoothed_preds, stemmed_preds)
significance = "significant" if p_value < 0.05 else "not significant"
print(f"results using stemming are {significance} with respect to smoothed NB with no stemming")

# Q4.2
print("--- determining the number of features before/after stemming ---")
NB.train(stemmed_corpus.train) # Retrain on full training data before counting number of features
num_stemmed_features = len(NB.vocabulary)
print(f"Number of features without stemming: {num_non_stemmed_features}")
print(f"Number of features with stemming: {num_stemmed_features}")

--- stemming corpus ---
--- cross-validating NB using stemming ---
Accuracy: 0.814
Std. Dev: 0.026
results using stemming are not significant with respect to smoothed NB with no stemming
--- determining the number of features before/after stemming ---
Number of features without stemming: 52555
Number of features with stemming: 32404


In [12]:
# question Q5.0
# cross-validate model using smoothing and bigrams
print("--- cross-validating naive bayes using smoothing and bigrams ---")
NB=NaiveBayesText(smoothing=True,bigrams=True,trigrams=False,discard_closed_class=False)
NB.crossValidate(corpus)
smoothed_and_bigram_preds=NB.predictions
print(f"Accuracy: {NB.getAccuracy():.2f}") 
print(f"Std. Dev: {NB.getStdDeviation():.2f}")

# get number of bigram features
NB.train(corpus.train)
num_bigram_features = len(NB.vocabulary)

# see if bigrams significantly improves results on smoothed NB only
p_value=signTest.getSignificance(smoothed_preds,smoothed_and_bigram_preds)
signifance = "significant" if p_value < 0.05 else "not significant"
print(f"results using smoothing and bigrams are {signifance} with respect to smoothing only")

# cross-validate model using smoothing and bigrams and trigrams
print("--- cross-validating naive bayes using smoothing and bigrams and trigrams  ---")
NB=NaiveBayesText(smoothing=True,bigrams=True,trigrams=True,discard_closed_class=False)
NB.crossValidate(corpus)
smoothed_and_bigram_and_trigrams_preds=NB.predictions
print(f"Accuracy: {NB.getAccuracy():.2f}")
print(f"Std. Dev: {NB.getStdDeviation():.2f}")

# get number of trigram features
NB.train(corpus.train)
num_trigram_features = len(NB.vocabulary)

# see if bigrams+trigrams significantly improves results on smoothed NB only
p_value=signTest.getSignificance(smoothed_preds,smoothed_and_bigram_and_trigrams_preds)
significance = "significant" if p_value < 0.05 else "not significant"
print(f"results using smoothing and bigrams and trigrams are {significance} with respect to smoothing only")


# Q5.1
print("--- determining the number of features with/without bigrams ---")
num_unigram_features = num_non_stemmed_features
print(f"Number of features with just unigrams smoothing: {num_unigram_features}")
print(f"Number of features with unigrams and bigrams: {num_bigram_features}")
print(f"Number of features with unigrams, bigrams and trigrams: {num_trigram_features}")

--- cross-validating naive bayes using smoothing and bigrams ---
Accuracy: 0.83
Std. Dev: 0.03
results using smoothing and bigrams are not significant with respect to smoothing only
--- cross-validating naive bayes using smoothing and bigrams and trigrams  ---


In [ ]:
# TODO Q6 and 6.1
print("--- classifying reviews using SVM 10-fold cross-eval ---")

In [ ]:
# TODO Q7
print("--- adding in POS information to corpus ---")
print("--- training svm on word+pos features ----")
print("--- training svm discarding closed-class words ---")

In [ ]:
# question 8.0
print("--- using document embeddings ---")